# Experiment analysis: Size Varying sprite discs
## animals are tectum MTZ ablated SAGFF(LF)81C with UAS:NTR-mcherry and controls (no mcherry expression visible)

In [ ]:
import socket
import os
import pandas as pd

In [ ]:
if 'startDirMaster' not in locals():
    startDirMaster=os.getcwd()

propsFn=startDirMaster+'\\props.csv'

props=pd.read_csv(propsFn, header=None, index_col=0, squeeze=True,delim_whitespace=True).to_dict()
#print(props)
props

codeDir = '..\\jlsocialbehavior'
    

if 'NB_path' not in locals():
    NB_path = os.getcwd()
else:
    os.chdir(NB_path)
    

base=props['BaseDir']
#artDir=props['ArtDir']
RawDataDir = os.path.join(base,props['RawDataDir'])+'\\'+'behavior\\'
ProcessingDir = os.path.join(base,props['ProcessingDir'])+'\\'
outputDir = os.path.join(base,props['outputDir'])+'\\'
FigureDir = os.path.join(base,props['FigureDir'])+'\\'

expFile=props['allExpBehFn']
anFile=props['allAnBehFn']


props

print('baseFolder = ' + base)



os.chdir(codeDir)
props

In [ ]:
%config InteractiveShellApp.pylab_import_all = False
%matplotlib inline
%pylab inline
%reload_ext autoreload
%autoreload 2

import sys
import fnmatch
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame, Series
import seaborn as sns
import glob
from datetime import datetime
from scipy import stats
import statsmodels.stats.api as sms

import functions.matrixUtilities_joh as mu
import functions.notebookHelper as nh
import matplotlib.pyplot as plt
import models.experiment as xp
import models.experiment_set as es
import functions.paperFigureProps as pfp
import functions.peakdet as pkd

palSimplePoint=['k','r']
treatName=np.array(['RFP+MTZ+','RFP+MTZ-','RFP-MTZ+','RFP-MTZ-'])
treatNameSimple=np.array(['Ablation','Control','Control','Control'])

pfp.paper()
plt.rcParams['svg.fonttype'] = 'none'

import matplotlib as mpl
mpl.rcParams['svg.fonttype'] = 'none'

inToCm=2.54

Pre-Analyze all experiments only if necessary, this takes a couple of minutes! Experiment summary csv files are saved to disk.

In [ ]:
info=pd.read_excel(expFile)
ix=(info.stimulusProtocol=='81c')#&(info.date.isin(['2019-08-05','2019-08-06','2019-09-25','2019-10-04']))
info=info[ix].reset_index(drop=True)
info

In [ ]:
infoAn=pd.read_excel(anFile)
infoAn.tail()

In [ ]:
# collect meta information and save to new csv file for batch processing

aviPath=[]
posPath=[]
PLPath=[]
expTime = []
birthDayAll=[]
anIDsAll=[]
camHeightAll=[]

#camHeight=[105,180] #for arena up,dn

for index,row in info.iterrows():

    startDir=RawDataDir+row.path+'\\'
    if not os.path.isdir(startDir):
        print(startDir)
        raise FileNotFoundError 
    
    posPath.append(glob.glob(startDir+'PositionTxt*.txt')[0])
    PLPath.append(glob.glob(startDir+'PL*.txt')[0])
    
    head, tail = os.path.split(posPath[-1])
    currTime=datetime.strptime(tail[-23:-4], '%Y-%m-%dT%H_%M_%S')
    expTime.append(currTime)
    
    #camHeightAll.append(camHeight[('_dn_' in head)*1])
    
    anNrs=row.anNr #Note that anNrs are 1 based!
    if ':' in anNrs:
        a,b=anNrs.split(sep=':')
        anNrs=np.arange(int(a),int(b)+1)
    else:
        anNrs=np.array(anNrs.split()).astype(int)
        
    anIDs=anNrs #-1 no more 0-based since using pandas merge to find animal numbers
    anIDsAll.extend(anIDs)

    bd=infoAn[infoAn.anNr.isin(anIDs)].bd.values
    #bd=infoAn.bd.values[anIDs-1] #a bit dirty to use anIDs directly here. Should merge
    birthDayAll.append(' '.join(list(bd)))

#info['camHeight']=camHeightAll
info['txtPath']=posPath
info['pairList']=PLPath
info['aviPath']='default'
info['birthDayAll']=birthDayAll
info['epiDur'] = 5      # duration of individual episodes (default: 5 minutes)
info['episodes'] = -1   # number of episodes to process: -1 to load all episodes (default: -1)
info['inDish'] = 10#np.arange(len(posPath))*120     # time in dish before experiments started (default: 10)
info['arenaDiameter_mm'] = 100 # arena diameter (default: 100 mm)
info['minShift'] = 60 # minimum number of seconds to shift for control IAD
info['episodePLcode'] = 0 # flag if first two characters of episode name encode animal pair matrix (default: 0)
info['recomputeAnimalSize'] = 0 # flag to compute animals size from avi file (takes time, default: 1)
info['SaveNeighborhoodMaps'] = 1 # flag to save neighborhood maps for subsequent analysis (takes time, default: 1)
info['computeLeadership'] = 0 # flag to compute leadership index (takes time, default: 1)
info['ComputeBouts'] = 1 # flag to compute swim bout frequency (takes time, default: 1)
#info['set'] = np.arange(len(posPath))   # experiment set: can label groups of experiments (default: 0)
info['ProcessingDir']=ProcessingDir
info['outputDir']=outputDir
info['expTime']=expTime

In [ ]:
csvFile=os.path.join(ProcessingDir,'processingSettings.csv')
info.to_csv(csvFile,encoding='utf-8')
info.tail()

In [ ]:
rereadData=1
if rereadData:
    def readExperiment(keepData=True):
        tmp=es.experiment_set(csvFile=csvFile,MissingOnly=False)
        if keepData:
            return tmp
        else:
            return 1

    expSet=readExperiment(keepData=True)

In [ ]:
csvPath = []
for f in [mu.splitall(x)[-1][:-4] for x in info.txtPath]:
    csvPath.append(glob.glob(ProcessingDir+f+'*siSummary*.csv')[0])

df=pd.DataFrame()
i=0
for fn in csvPath:
    print(fn)
    tmp=pd.read_csv(fn,index_col=0,sep=',')
    tmp.animalSet=i
    tmp.animalIndex=tmp.animalIndex+((i)*15)
    tmp['anIx']=tmp.animalIndex
    tmp.animalIndex=np.array(anIDsAll)[tmp.animalIndex]
    df=pd.concat([df,tmp])
    i+=1
df['episode']=[x.strip().replace('_','') for x in df['episode']]
df=pd.merge(df,infoAn[['anNr','line','group']],left_on='animalIndex',right_on='anNr',how='left')
df=pd.merge(df,info[['date']],left_on='animalSet',right_on=info.index,how='left')

print('df shape',df.shape)
df.tail(10)
df.group=df.group.astype('int')

In [ ]:
df.info()

In [ ]:
dfPlot=(df.groupby(['inDishTime','episode']).si.agg(['mean','std'])
    .unstack()
    .stack(dropna=True)
    .reset_index())

dfPlot.head()


In [ ]:
fig, axes = plt.subplots(figsize=(6, 3))
g=sns.scatterplot(data=dfPlot,x='inDishTime',hue='episode',y='mean')
plt.xlim([0,10*60])
plt.ylim([-.05,.4])
plt.xlabel('Time (Minutes)')
plt.ylabel('Attraction')
plt.title('Mean attraction, all animals')
#plt.legend(title='Stimulus dot motion')
#plt.legend(labels=['Continuous motion','Bout motion','Continuous + loom','Bout + loom'],
#          title='Stimulus dot motion')

#new_title = 'Stimulus dot motion'
#g.legend_.set_title(new_title)
# replace labels
plt.legend(ncol=1,handletextpad=0,bbox_to_anchor=(1, 1),loc='upper left')
plt.axhline(0)

sns.despine()

figPath=base+'ScreenTimeCourse.png'
#plt.savefig(figPath,bbox_inches='tight')


In [ ]:
idx=(df['inDishTime']<220) & (df['inDishTime']>45)
episodeNames=df['episode'].unique()
dfDR=df[idx]
col=df.groupby('animalIndex').group.first().values#treatment


In [ ]:
social=df[idx].groupby(['group','episode','animalIndex']).avgSpeed.mean().reset_index()
social['treatName']=treatName[social.group.values.astype('int')]
ax=sns.swarmplot(data=social,x='episode',hue='treatName',y='avgSpeed',zorder=1,linewidth=1,edgecolor='gray')
#sns.swarmplot(data=dfSIs,x='condition',y='si',zorder=1,linewidth=1,edgecolor='gray',color='white')


sns.pointplot(x="episode", y="avgSpeed", hue='treatName',data=social,ci=None,zorder=100,scale=2)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:4], labels[:4])
plt.xlabel('dot size [mm]')
plt.ylabel('average Speed [mm/sec]')
plt.xticks(range(7),['0','0.5','1','2','4','8','16'])
plt.axhline(0,ls=':',color='gray')
sns.despine()

In [ ]:
lAll=[]
for f in posPath:
    head, tail = os.path.split(f)
    search=head+'\PositionTxt*.txt'
    currTxt=glob.glob(search)[0]
    npyFileOut=currTxt[:-4]+'MapData.npy'
    head, tail = os.path.split(npyFileOut)
    npyFileOut=ProcessingDir+tail
    print('loading:',npyFileOut)
    lAll.append(np.load(npyFileOut))

In [ ]:
allMaps=np.concatenate(tuple(lAll),axis=0)#[idx]
allMaps.shape

In [ ]:
levels=df['episode'].unique()
treat=df['group'].unique()
avgT=np.zeros((len(treat),len(levels),allMaps.shape[3],allMaps.shape[4]))
avgTC=np.zeros((len(treat),len(levels),allMaps.shape[3],allMaps.shape[4]))

In [ ]:
#tmp=np.array([np.array([p.animals[0].ts.neighborMat() for p in e.pair]) for e in expSet.experiments])
#tmp2=np.concatenate(tmp)

In [ ]:
for t in treat:
    for i in range(len(levels)):
        ix=np.where((df['episode']==levels[i]) & (df['group']==t)&idx)[0]
        avgT[t,i,:,:]=allMaps[ix,0,0,:,:].mean(axis=0)
        avgTC[t,i,:,:]=allMaps[ix,0,1,:,:].mean(axis=0)

In [ ]:
import skimage
blurred = skimage.filters.gaussian(
    avgT[t,i,:,:], sigma=(3, 3))
plt.imshow(blurred,cmap='gray')

In [ ]:
from skimage.measure import EllipseModel
from matplotlib.patches import Ellipse

In [ ]:
from mpl_toolkits.axes_grid1 import AxesGrid
import matplotlib
def shiftedColorMap(cmap, start=0, midpoint=0.5, stop=1.0, name='shiftedcmap'):
    '''
    Function to offset the "center" of a colormap. Useful for
    data with a negative min and positive max and you want the
    middle of the colormap's dynamic range to be at zero

    Input
    -----
      cmap : The matplotlib colormap to be altered
      start : Offset from lowest point in the colormap's range.
          Defaults to 0.0 (no lower ofset). Should be between
          0.0 and `midpoint`.
      midpoint : The new center of the colormap. Defaults to 
          0.5 (no shift). Should be between 0.0 and 1.0. In
          general, this should be  1 - vmax/(vmax + abs(vmin))
          For example if your data range from -15.0 to +5.0 and
          you want the center of the colormap at 0.0, `midpoint`
          should be set to  1 - 5/(5 + 15)) or 0.75
      stop : Offset from highets point in the colormap's range.
          Defaults to 1.0 (no upper ofset). Should be between
          `midpoint` and 1.0.
    '''
    cdict = {
        'red': [],
        'green': [],
        'blue': [],
        'alpha': []
    }

    # regular index to compute the colors
    reg_index = np.linspace(start, stop, 257)

    # shifted index to match the data
    shift_index = np.hstack([
        np.linspace(0.0, midpoint, 128, endpoint=False), 
        np.linspace(midpoint, 1.0, 129, endpoint=True)
    ])

    for ri, si in zip(reg_index, shift_index):
        r, g, b, a = cmap(ri)

        cdict['red'].append((si, r, r))
        cdict['green'].append((si, g, g))
        cdict['blue'].append((si, b, b))
        cdict['alpha'].append((si, a, a))

    newcmap = matplotlib.colors.LinearSegmentedColormap(name, cdict)
    plt.register_cmap(cmap=newcmap)

    return newcmap

In [ ]:
import matplotlib.gridspec as gridspec

pfp.paper()
inToCm=2.54

outer = gridspec.GridSpec(2, 2, width_ratios = [5,.1], wspace = 0.05) 
#make nested gridspecs
gs2 = gridspec.GridSpecFromSubplotSpec(4, 7, subplot_spec = outer[0])
gs3 = gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec = outer[1])
#gs4 = gridspec.GridSpecFromSubplotSpec(2, 7, subplot_spec = outer[2])

fig = plt.figure(figsize=(11/inToCm,11/inToCm))
axes = [fig.add_subplot(gs2[i]) for i in range(28)]
axesCB=[fig.add_subplot(gs3[i]) for i in range(1)]

axesSP=fig.add_subplot(outer[2])

m=np.nanpercentile(avgT,99)
orig_cmap = matplotlib.cm.coolwarm
cmap=shiftedColorMap(orig_cmap,midpoint=1-(m/(m+1)))
trLab=treatName
pal=['r','.6','.4','k']
for an in treat:
    for i in range(len(levels)):
        ind=i+(7*an)
        im = axes[ind].imshow(avgT[an,i,:,:],clim=[0,m],extent=[-31,31,-31,31],origin='lower',cmap=cmap)
        axes[ind].tick_params(axis='y', which='both',length=0)
        axes[ind].tick_params(axis='x', which='both',length=0)
        axes[ind].set_xticks([])
        axes[ind].set_yticks([])
        axes[ind].spines['top'].set_color('white')
        axes[ind].spines['bottom'].set_color('white')
        axes[ind].spines['left'].set_color('white')
        axes[ind].spines['right'].set_color('white')
        #axes[an,i].set_title('a:'+str(an)+trLab[an]+ 's:'+levels[i][-2:])
        #if an==1:
            #axes[ind].set_xlabel(int(levels[i][-2:])/2.)
        if i==0:
            axes[ind].set_ylabel(trLab[an],color=pal[an],rotation=0, labelpad=20)
            
        if (i==5)&(an==0):
            axes[ind].set_title('Neighbor density')

#plt.colorbar(im)
cbar=plt.colorbar(im,cax=axesCB[0],ticks=np.round([0,1,m-0.2]))

plt.subplots_adjust(wspace=0, hspace=0.1)

social=df[idx].groupby(['group','episode','animalIndex']).si.mean().reset_index()
social['xpretty']=["{:g}".format(int(ss[-2:])/2.) for ss in social.episode]
#dfPQ['xpretty']=["{:g}".format(int(ss[-2:])/2.) for ss in dfPQdiff.episode]

#treatName=np.array(['CTR+MTZ','C81+MTZ'])
social['treatName']=treatName[social.group.values.astype('int')]
#sns.swarmplot(data=social,x='xpretty',hue='treatName',y='si',
#              zorder=1,linewidth=1,edgecolor='gray',ax=axesSP,palette=pal,alpha=0.7)
#sns.swarmplot(data=dfSIs,x='condition',y='si',zorder=1,linewidth=1,edgecolor='gray',color='white')


sns.pointplot(x="xpretty", 
              y="si", 
              hue='treatName',
              data=social,
              dodge=.3,
              scale=0.5,
              ax=axesSP,
              palette=pal,
              linewidth=.5,
              edgecolor='gray',
              linestyles=':',
             ci='sd')

plt.setp(axesSP.collections, sizes=[20],zorder=-100)

axesSP.spines['top'].set_color('white')
axesSP.spines['bottom'].set_color('white')
#axesSP.spines['left'].set_color('white')
axesSP.spines['right'].set_color('white')
axesSP.tick_params(axis='x', which='both',length=0)
#axes[ind].set_xticks([])
axesSP.set_yticks([0,.1,.2,.3,.4])
axesSP.yaxis.tick_left()
axesSP.set_xlabel('Dot diameter (mm)')
axesSP.set_ylabel('Attraction')
handles, labels = axesSP.get_legend_handles_labels()
axesSP.legend(handles[:4], labels[:4],loc='upper left')
#plt.xlabel('dot size [mm]')
#plt.ylabel('attraction')
#plt.xticks(range(7),['0','0.5','1','2','4','8','16'])
axesSP.axhline(0,ls=':',color='k')
#sns.despine()

#fig.text(0.5, -0.11, 'dot diameter [mm]', ha='center')



figPath=FigureDir+'5_MTZ_mapsAndAttraction.svg'
plt.savefig(figPath,bbox_inches='tight')

In [ ]:
i=4
t=27

ix=np.where((df['episode']==levels[i]) & (df['anIx']==t)&idx)[0]
currentMap=np.nanmean(allMaps[ix,0,0,:,:],axis=0)
#avgTCa[t,i,:,:]=np.nanmean(allMaps[ix,0,1,:,:],axis=0)

blurred = skimage.filters.gaussian(
currentMap, sigma=(3, 3))

nSamp=25 #number of samples around circle
cx,cy=31,31 #center of circle
l=cx-2 #length of intensity profiles outward from center

samp=np.linspace(0,2*np.pi,nSamp)
fig,axes=plt.subplots(2,2,figsize=(11/inToCm,11/inToCm),sharey='row')

m=5
cmap=shiftedColorMap(orig_cmap,midpoint=1-(m/(m+1)))

im=axes[0,0].imshow(currentMap,cmap=cmap)
axes[0,1].imshow(blurred,cmap='gray')

maxAll=np.zeros((nSamp,7))
profileAll=np.zeros((nSamp,l))

for j,s in enumerate(samp):
    a=np.sin(s)*l+cx
    b=np.cos(s)*l+cy
    profile=skimage.measure.profile_line(blurred, (31,31), (a,b), linewidth=5,mode='reflect')
    profileAll[j]=profile[:l]
    axes[1,0].plot(profile)
    axes[0,1].plot([31,b],[31,a],'.-',ms=4)
    r=np.argmax(profile)
    my=np.sin(s)*r+cx
    mx=np.cos(s)*r+cy
    mi=np.nanmax(profile)
    attraction=r*mi
    aversion=attraction-profile[:r].sum()
    dip=mi-np.mean(profile[:2])
    maxAll[j]=[mx,my,r,mi,attraction,aversion,dip]
    axes[0,1].plot(mx,my,'k.',ms=5)
    
#x = radiusAll[:, 0]
#y = radiusAll[:, 1]

#ell = EllipseModel()
#ell.estimate(maxAll[:,:2])

#xc, yc, a, b, theta = ell.params
#ell_patch = Ellipse((xc, yc), 2*a, 2*b, theta, edgecolor='green', facecolor='none')

#axes[0,1].add_patch(ell_patch)
axes[1,0].plot(maxAll[:,2],maxAll[:,3],'.k',ms=5)
axes[1,0].set_ylim([0,2])
mp=profileAll.mean(axis=0)
mm=np.argmax(mp)
m=np.max(mp)
axes[1,1].fill_between(range(mm+1),mp[mm],
                       lw=0,
                       facecolor='b')
axes[1,1].fill_between(range(mm+1),mp[:mm+1],
                       lw=0,
                       facecolor='r')

axes[1,1].plot(mp,c='k')
axes[1,1].plot(mm,m,'.k',ms=5)
axes[1,1].set_ylim([0,3])

axes[1,0].set_ylabel('Neighbor density')
axes[1,0].set_xlabel('Radius (mm)')
axes[1,1].set_xlabel('Radius (mm)')

axes[1,0].axhline(1,ls=':',c='gray')
axes[1,1].axhline(1,ls=':',c='gray')
axes[0,0].axis('off')
axes[0,1].axis('off')
axes[0,0].plot([2,22],[63,63],'k',lw=3,solid_capstyle='butt')

cax = fig.add_axes([0.48, 0.57, 0.01, 0.1])
cbar=plt.colorbar(im,cax=cax,ticks=np.round([0,1,5]))


sns.despine()

figPath=FigureDir+'S7_computeAversion.svg'
plt.savefig(figPath,bbox_inches='tight')

In [ ]:
df['groupSimple']=df.group
df.loc[df.groupSimple>0,'groupSimple']=1

avgTs=np.zeros((2,len(levels),allMaps.shape[3],allMaps.shape[4]))
avgTCs=np.zeros((2,len(levels),allMaps.shape[3],allMaps.shape[4]))
treatSimple=df['groupSimple'].unique()
for t in treatSimple:
    for i in range(len(levels)):
        ix=np.where((df['episode']==levels[i]) & (df['groupSimple']==t)&idx)[0]
        avgTs[t,i,:,:]=allMaps[ix,0,0,:,:].mean(axis=0)
        avgTCs[t,i,:,:]=allMaps[ix,0,1,:,:].mean(axis=0)

In [ ]:
def drawArrow(ax,length=40,pos=(100,100),labels=['A','L'],fontSize=8):
    ax.annotate( labels[0], xy=(pos[0]-length, pos[1]),
              xytext=(pos[0]-length, pos[1]-length) , 
                va = "bottom", 
                ha="center",
                color='w',
                fontsize=fontSize,
               arrowprops=dict(color='w',
                              arrowstyle='<-',
                               lw=1,
                              shrinkB=0),
                bbox=dict(pad=-5, facecolor="none", edgecolor="none")
               )
    ax.annotate( labels[1], xy=(pos[0]-length, pos[1]),
              xytext=(pos[0], pos[1]) , 
                va = "center", 
                ha="center",
                color='w',
                fontsize=fontSize,
               arrowprops=dict(color='w',
                              arrowstyle='<-',
                               lw=1,
                               shrinkA=0,
                              shrinkB=0),
                bbox=dict(pad=-0, facecolor="none", edgecolor="none")
               )

In [ ]:
ixPl=(df.episode=='knot04')&idx

In [ ]:
treatName[[3,2,1,0]]

In [ ]:
pal

In [ ]:
pfp.paper()
fig, ax = plt.subplots(figsize=(2.5/inToCm,5/inToCm))
pal=np.array(pal)
si=df[ixPl].groupby(['group','animalIndex']).si.mean().reset_index()
#treatName=np.array(['CTR+MTZ','C81+MTZ'])
si['treatName']=treatName[speed.group.values.astype('int')]
sns.pointplot(y="si", 
            x='treatName',
              hue='treatName',
              order=treatName[[3,2,1,0]],
              hue_order=treatName[[3,2,1,0]],
            data=si,
            ax=ax,
                            scale=.7,
              #color='k',
              zorder=100,
              join=False,
            palette=pal[[3,2,1,0]],
            linewidth=2,
            ci='sd')

sns.stripplot(y="si", 
            x='treatName',
            data=si,
            ax=ax,
            palette=pal[[3,2,1,0]],
              order=treatName[[3,2,1,0]],
              edgecolor='w',
            alpha=.5,
              linewidth=1,
              zorder=-100
            )




plt.xlabel('')
plt.xticks([])
plt.ylabel('Attraction')

l=ax.legend()
h,l = ax.get_legend_handles_labels()
ax.legend(h,l,frameon=False,ncol=1,
         loc=1,bbox_to_anchor=(1.7, 1),title='',
                        labelspacing=0,
               borderpad=0,
               handlelength=0,
               columnspacing=2,)   # <<<<<<<< This is where the magic happens

plt.ylim([-.1,.6])
ax.axhline(0,ls=':',color='k')
sns.despine()
plt.subplots_adjust(wspace=0, hspace=0)

figPath=FigureDir+'S7_81cControlSplit.svg'
plt.savefig(figPath,bbox_inches='tight')

In [ ]:
pfp.paper()
fig, ax = plt.subplots(figsize=(3/inToCm,8/inToCm))

speed=df[idx].groupby(['group','animalIndex']).avgSpeed_smooth.mean().reset_index()
#treatName=np.array(['CTR+MTZ','C81+MTZ'])
speed['treatName']=treatName[speed.group.values.astype('int')]
sns.pointplot(y="avgSpeed_smooth", 
            x='treatName',
            data=speed,
            ax=ax,
              #color='k',
              zorder=100,
              join=False,
            palette=pal,
            linewidth=2,
            ci='sd')

sns.stripplot(y="avgSpeed_smooth", 
            x='treatName',
            data=speed,
            ax=ax,
            palette=pal,
              edgecolor='w',
            alpha=.5,
              linewidth=1,
              zorder=-100
            )

#sns.swarmplot(data=dfSIs,x='condition',y='si',zorder=1,linewidth=1,edgecolor='gray',color='white')

# Select which box you want to change    
for i,artist in enumerate(ax.artists):
# Change the appearance of that box
    #mybox.set_linecolor('red')
    artist.set_edgecolor('k')
    for j in range(i*6,i*6+6):
        line = ax.lines[j]
        line.set_color('k')
        line.set_mfc('k')
        line.set_mec('k')

#sns.swarmplot(data=social,x='treatName',y='avgSpeed_smooth',zorder=100,linewidth=1,ax=ax,palette=pal,alpha=0.7,edgecolor='k')

plt.xlabel('')
plt.xticks([])
plt.ylabel('Average speed \n (mm/sec)')
#plt.xticks(range(7),['0','0.5','1','2','4','8','16'])
#plt.axhline(0,ls=':',color='gray')
#for tick in ax.xaxis.get_major_ticks():
#    tick.label.set_rotation(45)
plt.ylim([0,10])
sns.despine()
plt.subplots_adjust(wspace=0, hspace=0)

figPath=FigureDir+'5_MTZ_AvgSpeed.svg'
plt.savefig(figPath,bbox_inches='tight')

In [ ]:
speed.head()

In [ ]:
pfp.paper()
fig, ax = plt.subplots(figsize=(2/inToCm,4.5/inToCm))
palSimpleBar=['r','gray']


speed['treatNameSimple']=treatNameSimple[speed.group.values.astype('int')]
sns.pointplot(y="avgSpeed_smooth", 
            x='treatNameSimple',
            hue='treatNameSimple',
            data=speed,
            ax=ax,
              join=False,
              color='k',
              legend=True,
              order=['Control','Ablation'],
            #palette=palSimpleBar,
            linewidth=2,
            ci='sd',
             zorder=100)

sns.stripplot(y="avgSpeed_smooth", 
            x='treatNameSimple',
              hue='treatNameSimple',
              order=['Control','Ablation'],
            data=speed,
            ax=ax,
            palette=palSimpleBar,
              alpha=.5,
              zorder=-100
            #linewidth=2,
            )



plt.xlabel('')
plt.xticks([])
plt.ylabel('Speed (mm/s)')

plt.ylim([0,10])
sns.despine()
plt.subplots_adjust(wspace=0, hspace=0)
ax.set_yticks([0,5,10])

l=ax.legend()
h,l = ax.get_legend_handles_labels()
ax.legend(h[2:][::-1], ['Abl','Ctr'][::-1],frameon=False,ncol=2,
         loc=1,bbox_to_anchor=(1, 1.3),title='',
                        labelspacing=0,
               borderpad=0,
               handlelength=0,
               columnspacing=2,)   # <<<<<<<< This is where the magic happens

ax.text(0,10,'p=0.018')
figPath=FigureDir+'5_MTZ_AvgSpeedSimple.svg'
plt.savefig(figPath,bbox_inches='tight')

In [ ]:
pfp.paper()
fig, ax = plt.subplots(figsize=(2/inToCm,4.5/inToCm))
palSimpleBar=['r','gray']
bout=df[idx].groupby(['group','animalIndex']).boutDur.mean().reset_index()
bout['treatNameSimple']=treatNameSimple[bout.group.values.astype('int')]
bout['boutRate']=1/bout['boutDur']
sns.pointplot(y="boutDur", 
            x='treatNameSimple',
            hue='treatNameSimple',
            data=bout,
            ax=ax,
              join=False,
              color='k',
            #palette=palSimpleBar,
            linewidth=2,
            ci='sd',
             zorder=100)

sns.stripplot(y="boutDur", 
            x='treatNameSimple',
              hue='treatNameSimple',
            data=bout,
            ax=ax,
            palette=palSimpleBar,
              alpha=.5,
              zorder=-100
            #linewidth=2,
            )

#sns.swarmplot(data=dfSIs,x='condition',y='si',zorder=1,linewidth=1,edgecolor='gray',color='white')

# Select which box you want to change    
for i,artist in enumerate(ax.artists):
# Change the appearance of that box
    #mybox.set_linecolor('red')
    artist.set_edgecolor('k')
    for j in range(i*6,i*6+6):
        line = ax.lines[j]
        line.set_color('k')
        line.set_mfc('k')
        line.set_mec('k')

#sns.swarmplot(data=social,x='treatName',y='avgSpeed_smooth',zorder=100,linewidth=1,ax=ax,palette=pal,alpha=0.7,edgecolor='k')

plt.xlabel('')
plt.xticks([])
plt.ylabel('Bout duration\n (s)')

plt.ylim([0,1.5])
sns.despine()
plt.subplots_adjust(wspace=0, hspace=0)

l=ax.legend().remove()
ax.text(0,1.5,'p=0.068')
figPath=FigureDir+'5_MTZ_BoutDurSimple.svg'
plt.savefig(figPath,bbox_inches='tight')

In [ ]:
import scipy.stats as sta

d2 = speed[speed['treatNameSimple'] == 'Control'].avgSpeed_smooth#iloc[:,2:]
d1 = speed[speed['treatNameSimple'] == 'Ablation'].avgSpeed_smooth#iloc[:,2:]
sta.ttest_ind(d1,d2)

In [ ]:
pfp.paper()
fig, ax = plt.subplots(figsize=(2/inToCm,4.5/inToCm))
palSimpleBar=['r','gray']

bout['treatNameSimple']=treatNameSimple[bout.group.values.astype('int')]
bout['boutRate']=1/bout['boutDur']
sns.pointplot(y="boutRate", 
            x='treatNameSimple',
            hue='treatNameSimple',
            data=bout,
            ax=ax,
              join=False,
              color='k',
            #palette=palSimpleBar,
            linewidth=2,
              order=['Control','Ablation'],
            ci='sd',
             zorder=100)

sns.stripplot(y="boutRate", 
            x='treatNameSimple',
              hue='treatNameSimple',
              order=['Control','Ablation'],
            data=bout,
            ax=ax,
            palette=palSimpleBar,
              alpha=.5,
              zorder=-100
            #linewidth=2,
            )

#sns.swarmplot(data=dfSIs,x='condition',y='si',zorder=1,linewidth=1,edgecolor='gray',color='white')

# Select which box you want to change    
for i,artist in enumerate(ax.artists):
# Change the appearance of that box
    #mybox.set_linecolor('red')
    artist.set_edgecolor('k')
    for j in range(i*6,i*6+6):
        line = ax.lines[j]
        line.set_color('k')
        line.set_mfc('k')
        line.set_mec('k')

#sns.swarmplot(data=social,x='treatName',y='avgSpeed_smooth',zorder=100,linewidth=1,ax=ax,palette=pal,alpha=0.7,edgecolor='k')

plt.xlabel('')
plt.xticks([])
plt.ylabel('Bout rate (Hz)')
#plt.xticks(range(7),['0','0.5','1','2','4','8','16'])
#plt.axhline(0,ls=':',color='gray')
#for tick in ax.xaxis.get_major_ticks():
#    tick.label.set_rotation(45)
plt.ylim([0,2.5])
sns.despine()
plt.subplots_adjust(wspace=0, hspace=0)

l=ax.legend().remove()
ax.text(0,2.5,'p=0.068')
figPath=FigureDir+'5_MTZ_BoutRateSimple.svg'
plt.savefig(figPath,bbox_inches='tight')

In [ ]:
import scipy.stats as sta

d2 = bout[bout['treatNameSimple'] == 'Control'].boutDur#iloc[:,2:]
d1 = bout[bout['treatNameSimple'] == 'Ablation'].boutDur#iloc[:,2:]
sta.ttest_ind(d1,d2)

In [ ]:
pfp.paper()

fig, axes = plt.subplots(nrows=4, ncols=7, sharex=True, sharey=True,figsize=(15/inToCm,10/inToCm))
m=np.nanpercentile(avgT,95)
trLab=treatName

quantRange=[20,40]


for an in treat:
    for i in range(len(levels)):
        profile=avgT[an,i,:,quantRange[0]:quantRange[1]].mean(axis=1)
        axes[an,i].plot(profile,np.arange(profile.shape[0])-30)
        #axes[an,i].set_title('a:'+str(an)+trLab[an]+ 's:'+levels[i][-2:],fontsize=10)
        axes[an,i].axhline(0,ls=':',color='gray')
        profile=avgTC[an,i,:,quantRange[0]:quantRange[1]].mean(axis=1)
        axes[an,i].plot(profile,np.arange(profile.shape[0])-30,ls=':',color='k')
        axes[an,i].axvline(0,color='k',linewidth=1)
        axes[an,i].axhline(quantRange[0]-30,color='gray',linewidth=1)
        axes[an,i].axhline(quantRange[1]-30,color='gray',linewidth=1)

        axes[an,i].set_ylim([-30,30])

In [ ]:
blurred.shape[0]*blurred.shape[1]

In [ ]:
def mapAversion(NeighborMap,nSamp=24,c=(31,31),l=29):
    nSamp=nSamp #number of samples around circle
    cx,cy=c[0],c[1] #center of circle
    l=l #length of intensity profiles outward from center

    samp=np.linspace(0,2*np.pi,nSamp)
    maxAll=np.zeros((nSamp,7))
    
    blurred = skimage.filters.gaussian(
    NeighborMap, sigma=(3, 3))
    
    profileAll=np.zeros((nSamp,l))

    for i,s in enumerate(samp):
        a=np.sin(s)*l+cx
        b=np.cos(s)*l+cy
        profile=skimage.measure.profile_line(NeighborMap, (31,31), (a,b), linewidth=5,mode='reflect')
        profileAll[i]=profile[:l]
        r=np.argmax(profile)
        my=np.sin(s)*r+cx
        mx=np.cos(s)*r+cy
        mi=np.nanmax(profile)
        attraction=r*mi
        aversion=attraction-profile[:r].sum()
        aversion=aversion/l
        dip=mi-np.mean(profile[:2])
        maxAll[i]=[mx,my,r,mi,attraction,aversion,dip]
        
    profile=np.mean(profileAll,axis=0)
    r=np.argmax(profile)
    my=np.sin(s)*r+cx
    mx=np.cos(s)*r+cy
    mi=np.nanmax(profile)
    attraction=r*mi
    aversion=attraction-profile[:r].sum()
    aversion=aversion/l
    dip=mi-np.mean(profile[:2])

    maxMean=[mx,my,r,mi,attraction,aversion,dip]
    
    return maxAll,maxMean,profileAll

In [ ]:
levels=df['episode'].unique()
Animals=df['animalIndex'].unique()

avgTa=np.zeros((len(Animals),len(levels),allMaps.shape[3],allMaps.shape[4]))
avgTCa=np.zeros((len(Animals),len(levels),allMaps.shape[3],allMaps.shape[4]))
dfPQ=pd.DataFrame()

#fig,ax=plt.subplots(20,7,figsize=(7,20),sharey=True,sharex=True)
#ax=ax.ravel()
j=0
for t,c in enumerate(col):
    for i in range(len(levels)):
        
        ix=np.where((df['episode']==levels[i]) & (df['anIx']==t)&idx)[0]
        avgTa[t,i,:,:]=np.nanmean(allMaps[ix,0,0,:,:],axis=0)
        avgTCa[t,i,:,:]=np.nanmean(allMaps[ix,0,1,:,:],axis=0)

In [ ]:
df.head()

In [ ]:
levels=df['episode'].unique()
Animals=df['animalIndex'].unique()

#avgTa=np.zeros((len(Animals),len(levels),allMaps.shape[3],allMaps.shape[4]))
#avgTCa=np.zeros((len(Animals),len(levels),allMaps.shape[3],allMaps.shape[4]))
dfPQ=pd.DataFrame()

#fig,ax=plt.subplots(20,7,figsize=(7,20),sharey=True,sharex=True)
#ax=ax.ravel()
j=0
for t,c in enumerate(col):
    for i in range(len(levels)):
        
        ix=np.where((df['episode']==levels[i]) & (df['anIx']==t)&idx)[0]
        currentMap=np.nanmean(allMaps[ix,0,0,:,:],axis=0)
        #avgTCa[t,i,:,:]=np.nanmean(allMaps[ix,0,1,:,:],axis=0)
        
        blurred = skimage.filters.gaussian(
        currentMap, sigma=(3, 3))
            
        #profile=avgTa[t,i,29:31,:].mean(axis=0)
        profile=blurred[quantRange[0]:quantRange[1],:].mean(axis=0)
        
        profileC=avgTCa[t,i,quantRange[0]:quantRange[1],:].mean(axis=0)
        profileSub=profile/profileC
        mid=profile[quantRange[0]:quantRange[1]].mean()
        high=np.nanpercentile(profile,95)
        mid1=28
        mid2=33
        center=np.nanmean(profile[mid1:mid2])
        ma=mapAversion(avgTa[t,i])
        #aversion=ma[0].mean(axis=0)[5]
        #radius=ma.mean(axis=0)[2]
        #dip2=ma.mean(axis=0)[6]
        aversion=ma[1][5]
        radius=ma[1][2]
        dip2=ma[1][6]
        if j<0:#140:
            ax[t,i].imshow(currentMap,cmap='gray',clim=(0,5))
            profile=np.median(ma[2],axis=0)
            ax[t,i].plot(profile)
            ax[t,i].axhline(high,c='r')
            ax[t,i].axhline(center,c='k')
            ax[t,i].set_title(levels[i]+str(t))
            j=j+1
        dip=high-center
        sub=profileSub[quantRange[0]:quantRange[1]].mean()
        
        #print t,i,{'an':t,'episode':levels[i],'treat':treatment[t],'lo':lo,'high':high}
        tmp=pd.DataFrame({'an':[t],
                          'episode':levels[i],
                          'treat':col[t],
                          'mid':[mid],
                          'high':[high],
                          'sub':[sub],
                          'center':[center],
                          'dip':[dip],
                          'dip2':[dip2],
                          'radius':[radius],
                         'aversion':[aversion]})
        dfPQ=pd.concat([dfPQ,tmp])
    #print(t,c,i,sum((df['episode']==levels[i]) & (df['anIx']==t)&idx))

In [ ]:
i=4
t=27

ix=np.where((df['episode']==levels[i]) & (df['anIx']==t)&idx)[0]
currentMap=np.nanmean(allMaps[ix,0,0,:,:],axis=0)
#avgTCa[t,i,:,:]=np.nanmean(allMaps[ix,0,1,:,:],axis=0)

blurred = skimage.filters.gaussian(
currentMap, sigma=(3, 3))
plt.imshow(blurred)

In [ ]:
ma=mapAversion(avgTa[t,i])

In [ ]:
def normGroup(x):
    return x.mid-x.mid.values[0]

In [ ]:
dfPQdiff=dfPQ.groupby(['an']).apply(normGroup).reset_index()
dfPQdiff=dfPQdiff.set_index('an')
#dfPQdiff=dfPQdiff.drop('mid',axis=1)
dfPQdiff.columns=dfPQ.episode.unique()
dfPQdiff=dfPQdiff.stack().reset_index()
dfPQdiff.columns=['an','episode','density']
dfPQdiff['treat']=col[dfPQdiff.an]


simple=np.array([0,1,1,1])

dfPQ['treatSimple']=simple[dfPQ['treat']]
dfPQdiff['treatSimple']=simple[dfPQdiff['treat']]
dfPQdiff.head()

In [ ]:
nameSimple=np.array(['Ablation','Control'])
dfPQdiff['gt']=nameSimple[dfPQdiff.treatSimple]
dfPQdiff['xpretty']=["{:g}".format(int(ss[-2:])/2.) for ss in dfPQdiff.episode]

dfPQ['gt']=nameSimple[dfPQdiff.treatSimple]
dfPQ['xpretty']=["{:g}".format(int(ss[-2:])/2.) for ss in dfPQdiff.episode]

dfPQdiff.head()

In [ ]:
levels=df['episode'].unique()
treat=df['group'].unique()



fig, axes = plt.subplots(nrows=4, ncols=7, sharex='col', sharey=True,figsize=(12, 5))
m=np.nanpercentile(avgT,95)
#trLab=['c','abl']
for an in treat:
    for i in range(len(levels)):
        profile=avgT[an,i,quantRange[0]:quantRange[1],:].mean(axis=0)
        high=np.nanpercentile(profile,90)
        mid1=28
        mid2=33
        mid=np.nanmean(profile[mid1:mid2])
        axes[an,i].plot(profile)
        axes[an,i].set_title('a:'+str(an)+trLab[an]+ 's:'+levels[i][-2:],fontsize=10)
        axes[an,i].axvline(mid1)
        axes[an,i].axvline(mid2)
        axes[an,i].axhline(high,c='r')
        axes[an,i].axhline(mid,c='k')

In [ ]:
colSimple=df.groupby('animalIndex').groupSimple.first().values#treatment
colSimple

In [ ]:
order=dfPQdiff['xpretty'].unique()
order

In [ ]:
avoid=dfPQ
avoid['xpretty']=["{:g}".format(int(ss[-2:])/2.) for ss in avoid.episode]
#dfPQ['xpretty']=["{:g}".format(int(ss[-2:])/2.) for ss in dfPQdiff.episode]

#treatName=np.array(['CTR+MTZ','C81+MTZ'])
avoid['treatName']=treatName[avoid.treat.values.astype('int')]
avoid['treatNameSimple']=treatNameSimple[avoid.treat.values.astype('int')]
avoid.head()

In [ ]:
dfPQ.head()

In [ ]:
sns.pointplot(data=dfPQ,
             x='xpretty',
             hue='treatNameSimple',
             y='dip',
             ci='sd',
             dodge=0.3,
             linestyles=':',
             palette=palSimplePoint)

In [ ]:
sns.pointplot(data=dfPQ,
             x='xpretty',
             hue='treatNameSimple',
             y='aversion',
             ci='sd',
             dodge=0.3,
             linestyles=':',
             palette=palSimplePoint)

In [ ]:
sns.pointplot(data=dfPQ,
             x='xpretty',
             hue='treatNameSimple',
             y='radius',
             ci='sd',
             dodge=0.3,
             linestyles=':',
             palette=palSimplePoint)

In [ ]:
sns.pointplot(data=dfPQ,
             x='xpretty',
             hue='treatNameSimple',
             y='dip2',
             ci='sd',
             dodge=0.3,
             linestyles=':',
             palette=palSimplePoint)

In [ ]:
for i,x in enumerate(dfPQdiff.xpretty.unique()):
    ix=(dfPQdiff.xpretty==x)&(dfPQdiff['gt']=='Ablation')
    a=dfPQdiff[ix].density
    ix=(dfPQdiff.xpretty==x)&(dfPQdiff['gt']=='Control')
    b=dfPQdiff[ix].density


In [ ]:

fig,ax = plt.subplots(figsize=(6/inToCm,5/inToCm))
sns.pointplot(data=dfPQ,
             x='xpretty',
             hue='gt',
             y='sub',
             ci='sd',
             dodge=0.3,
             linestyles=':',
             palette=palSimplePoint,
             order=order,
             hue_order=['Control','Ablation'])

sns.stripplot(data=dfPQ,
             x='xpretty',
             hue='gt',
             y='sub',
             dodge=0.0001,
              palette=palSimplePoint,
             #color='gray',
             alpha=0.1,
             zorder=-1000,
             order=order,
             hue_order=['Control','Ablation'])

plt.axhline(1,ls=':',color='gray')
plt.legend().remove()

#h,l = ax.get_legend_handles_labels()
#ax.legend(h[:2], l[:2],frameon=False)   # <<<<<<<< This is where the magic happens

ax.spines['bottom'].set_color('white')
ax.tick_params(axis='x', which='both',length=0)
plt.xlabel('Dot Size (mm)')
plt.ylabel('Center occupancy')
#plt.ylim([.05,10])
ax.set_yscale('log')

ax.text(0-.2,15,'ns')
ax.text(1-.2,15,'***')
ax.text(2-.2,15,'***')
ax.text(3-.2,15,'***')

ax.text(4-.2,15,'***')
ax.text(5-.2,15,'***')
ax.text(6-.2,15,'ns')

sns.despine()
figPath=FigureDir+'5_MTZ_DensityChange.svg'
plt.savefig(figPath,bbox_inches='tight')

In [ ]:
for i,x in enumerate(dfPQ.xpretty.unique()):
    ix=(dfPQ.xpretty==x)&(dfPQ['gt']=='Ablation')
    a=dfPQ[ix]['sub']
    ix=(dfPQ.xpretty==x)&(dfPQ['gt']=='Control')
    b=dfPQ[ix]['sub']
    print(stats.ttest_ind(a,b))

In [ ]:
N = 100
tt_df = pd.DataFrame({'group': np.append(['A'] * N, ['B'] * N),
                      'x': np.random.randn(2 * N)})
tt_df['y'] = tt_df['x'] + np.random.randn(2 * N)
tt_df.head()

In [ ]:
import matplotlib.gridspec as gridspec
#plt.rcParams['svg.fonttype'] = 'path'

outer = gridspec.GridSpec(3, 2, width_ratios = [5,.1], wspace = 0.05,height_ratios = [1,2,2]) 
#make nested gridspecs
gs2 = gridspec.GridSpecFromSubplotSpec(2, 7, subplot_spec = outer[0])
gs3 = gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec = outer[1])
gs4 = gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec = outer[2])

fig = plt.figure(figsize=(6/inToCm,10/inToCm))
axes = [fig.add_subplot(gs2[i]) for i in range(14)]
axesCB=[fig.add_subplot(gs3[i]) for i in range(1)]

axesSP=fig.add_subplot(outer[2])
axOc=fig.add_subplot(outer[4])

m=np.nanpercentile(avgT,99)
orig_cmap = matplotlib.cm.bwr
cmap=shiftedColorMap(orig_cmap,midpoint=1-(m/(m+1)))
trLab=treatName
palSimplePoint=['k','r']
reorder=[1,0]
for a in treatSimple:
    an=reorder[a]
    for i in range(len(levels)):
        ind=i+(7*a)
        im = axes[ind].imshow(avgT[an,i,:,:],
                              clim=[0,m],
                              extent=[-31,31,-31,31],
                              origin='lower',
                              cmap=cmap,
                             interpolation='none')
        axes[ind].tick_params(axis='y', which='both',length=0)
        axes[ind].tick_params(axis='x', which='both',length=0)
        axes[ind].set_xticks([])
        axes[ind].set_yticks([])
        axes[ind].spines['top'].set_color('white')
        axes[ind].spines['bottom'].set_color('white')
        axes[ind].spines['left'].set_color('white')
        axes[ind].spines['right'].set_color('white')

        if i==0:
            axes[ind].set_ylabel(treatNameSimple[an],color=pal[an],rotation=0, labelpad=20)
            
        if (i==5)&(a==0):
            axes[ind].set_title('Neighbor density')

cbar=plt.colorbar(im,cax=axesCB[0],ticks=np.round([0,1,m-0.2]))
plt.subplots_adjust(wspace=0, hspace=0.1)

social['treatNameSimple']=treatNameSimple[social.group.values.astype('int')]


sns.pointplot(x="xpretty", 
              y="si", 
              hue='treatNameSimple',
              data=social,
              dodge=.3,
              scale=0.5,
              ax=axesSP,
              palette=palSimplePoint,
              linewidth=.5,
              edgecolor='gray',
              linestyles=':',
             ci='sd',
             order=order,
             hue_order=['Control','Ablation'])

sns.stripplot(x="xpretty", 
              y="si", 
              hue='treatNameSimple',
              data=social,
              dodge=.01,
              ax=axesSP,
              palette=palSimplePoint,
             size=1,
             alpha=.1,
             order=order,
             hue_order=['Control','Ablation'])

plt.setp(axesSP.collections, sizes=[20],zorder=-100)

axesSP.spines['bottom'].set_color('white')
axesSP.tick_params(axis='x', which='both',length=0)
axesSP.set_yticks([0,.1,.2,.3,.4])
axesSP.yaxis.tick_left()
axesSP.set_xlabel('')
axesSP.set_ylabel('Attraction')
#handles, labels = axesSP.get_legend_handles_labels()

#axesSP.legend(handles[:2], 
#              labels[:2],
#              loc='upper left',
#              frameon=False,
#             borderaxespad=0.,
#             handletextpad=0.0)

axesSP.legend().remove()

axesSP.axhline(0,ls=':',color='k')

axesSP.text(0-.2,.5,'ns')
axesSP.text(1-.4,.5,'***')
axesSP.text(2-.4,.5,'***')
axesSP.text(3-.4,.5,'***')

axesSP.text(4-.4,.5,'***')
axesSP.text(5-.4,.5,'***')
axesSP.text(6-.4,.5,'***')
axesSP.set_xticks([])
sns.despine()
plt.setp(axesSP.get_xticklabels(), visible=False)

sns.pointplot(data=dfPQ,
             x='xpretty',
             hue='gt',
             y='sub',
             ci='sd',
             dodge=0.3,
             linestyles=':',
             palette=palSimplePoint,
             order=order,
              ax=axOc,
             hue_order=['Control','Ablation'])

sns.stripplot(data=dfPQ,
             x='xpretty',
             hue='gt',
             y='sub',
             dodge=0.0001,
              palette=palSimplePoint,
             #color='gray',
             alpha=0.1,
             zorder=-1000,
             order=order,
              ax=axOc,
             hue_order=['Control','Ablation'])

axOc.axhline(1,ls=':',color='gray')
axOc.legend().remove()

axOc.spines['bottom'].set_color('white')
axOc.tick_params(axis='x', which='both',length=0)
axOc.set_xlabel('Dot Size (mm)')
axOc.set_ylabel('Center occupancy')
axOc.set_yscale('log')
axOc.set_yticklabels(['','','.1','1','10'])

axOc.text(0-.2,15,'ns')
axOc.text(1-.2,15,'***')
axOc.text(2-.2,15,'***')
axOc.text(3-.2,15,'***')

axOc.text(4-.2,15,'***')
axOc.text(5-.2,15,'***')
axOc.text(6-.2,15,'ns')

sns.despine()

figPath=FigureDir+'5_MTZ_mapsAndAttractionSimple_both.svg'
plt.savefig(figPath,bbox_inches='tight')

plt.rcParams['svg.fonttype'] = 'none'

In [ ]:
dfPQ['subInv']=1./dfPQ['sub']
dfPQ

In [ ]:
import matplotlib.gridspec as gridspec
#plt.rcParams['svg.fonttype'] = 'path'

outer = gridspec.GridSpec(3, 2, width_ratios = [5,.1], wspace = 0.05,height_ratios = [1,2,2]) 
#make nested gridspecs
gs2 = gridspec.GridSpecFromSubplotSpec(2, 7, subplot_spec = outer[0])
gs3 = gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec = outer[1])
gs4 = gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec = outer[2])

fig = plt.figure(figsize=(6/inToCm,10/inToCm))
axes = [fig.add_subplot(gs2[i]) for i in range(14)]
axesCB=[fig.add_subplot(gs3[i]) for i in range(1)]

axesSP=fig.add_subplot(outer[2])
axOc=fig.add_subplot(outer[4])

m=np.nanpercentile(avgT,99)
orig_cmap = matplotlib.cm.bwr
cmap=shiftedColorMap(orig_cmap,midpoint=1-(m/(m+1)))
trLab=treatName

reorder=[1,0]
for a in treatSimple:
    an=reorder[a]
    for i in range(len(levels)):
        ind=i+(7*a)
        im = axes[ind].imshow(avgT[an,i,:,:],
                              clim=[0,m],
                              extent=[-31,31,-31,31],
                              origin='lower',
                              cmap=cmap,
                             interpolation='none')
        axes[ind].tick_params(axis='y', which='both',length=0)
        axes[ind].tick_params(axis='x', which='both',length=0)
        axes[ind].set_xticks([])
        axes[ind].set_yticks([])
        axes[ind].spines['top'].set_color('white')
        axes[ind].spines['bottom'].set_color('white')
        axes[ind].spines['left'].set_color('white')
        axes[ind].spines['right'].set_color('white')

        if i==0:
            axes[ind].set_ylabel(treatNameSimple[an],color=pal[an],rotation=0, labelpad=20)
            
        if (i==5)&(a==0):
            axes[ind].set_title('Neighbor density')

cbar=plt.colorbar(im,cax=axesCB[0],ticks=np.round([0,1,m-0.2]))
plt.subplots_adjust(wspace=0, hspace=0.1)

social['treatNameSimple']=treatNameSimple[social.group.values.astype('int')]


sns.pointplot(x="xpretty", 
              y="si", 
              hue='treatNameSimple',
              data=social,
              dodge=.3,
              scale=0.5,
              ax=axesSP,
              palette=palSimplePoint,
              linewidth=.5,
              edgecolor='gray',
              linestyles=':',
             ci='sd',
             order=order,
             hue_order=['Control','Ablation'])

sns.stripplot(x="xpretty", 
              y="si", 
              hue='treatNameSimple',
              data=social,
              dodge=.01,
              ax=axesSP,
              palette=palSimplePoint,
             size=1,
             alpha=.1,
             order=order,
             hue_order=['Control','Ablation'])

plt.setp(axesSP.collections, sizes=[20],zorder=-100)

axesSP.spines['bottom'].set_color('white')
axesSP.tick_params(axis='x', which='both',length=0)
axesSP.set_yticks([0,.1,.2,.3,.4])
axesSP.yaxis.tick_left()
axesSP.set_xlabel('')
axesSP.set_ylabel('Attraction')
#handles, labels = axesSP.get_legend_handles_labels()

#axesSP.legend(handles[:2], 
#              labels[:2],
#              loc='upper left',
#              frameon=False,
#             borderaxespad=0.,
#             handletextpad=0.0)

axesSP.legend().remove()

axesSP.axhline(0,ls=':',color='k')

axesSP.text(0-.2,.5,'ns')
axesSP.text(1-.4,.5,'***')
axesSP.text(2-.4,.5,'***')
axesSP.text(3-.4,.5,'***')

axesSP.text(4-.4,.5,'***')
axesSP.text(5-.4,.5,'***')
axesSP.text(6-.4,.5,'***')
axesSP.set_xticks([])
sns.despine()
plt.setp(axesSP.get_xticklabels(), visible=False)

sns.pointplot(data=dfPQ,
             x='xpretty',
             hue='gt',
             y='subInv',
             #ci='sd',
             dodge=0.3,
             linestyles=':',
             palette=palSimplePoint,
             order=order,
              ax=axOc,
             hue_order=['Control','Ablation'])

sns.stripplot(data=dfPQ,
             x='xpretty',
             hue='gt',
             y='subInv',
             dodge=0.0001,
              palette=palSimplePoint,
             #color='gray',
             alpha=0.1,
             zorder=-1000,
             order=order,
              ax=axOc,
             hue_order=['Control','Ablation'])

axOc.axhline(1,ls=':',color='gray')
axOc.legend().remove()

axOc.spines['bottom'].set_color('white')
axOc.tick_params(axis='x', which='both',length=0)
axOc.set_xlabel('Dot size (mm)')
axOc.set_ylabel('Short range \navoidance')
axOc.set_yscale('log')
axOc.set_yticklabels(['','','.1','1','10'])

axOc.text(0-.2,15,'ns')
axOc.text(1-.2,15,'***')
axOc.text(2-.2,15,'***')
axOc.text(3-.2,15,'***')

axOc.text(4-.2,15,'***')
axOc.text(5-.2,15,'***')
axOc.text(6-.2,15,'ns')

sns.despine()

figPath=FigureDir+'5_MTZ_mapsAndAttractionSimple_both_inv.svg'
plt.savefig(figPath,bbox_inches='tight')

plt.rcParams['svg.fonttype'] = 'none'

In [ ]:
for i,x in enumerate(dfPQ.xpretty.unique()):
    ix=(dfPQ.xpretty==x)&(dfPQ['gt']=='Ablation')
    a=dfPQ[ix]['aversion']
    ix=(dfPQ.xpretty==x)&(dfPQ['gt']=='Control')
    b=dfPQ[ix]['aversion']
    print(stats.mannwhitneyu(a,b))

In [ ]:
import matplotlib.gridspec as gridspec
#plt.rcParams['svg.fonttype'] = 'path'

outer = gridspec.GridSpec(3, 2, width_ratios = [5,.1], wspace = 0.05,height_ratios = [1,2,2]) 
#make nested gridspecs
gs2 = gridspec.GridSpecFromSubplotSpec(2, 7, subplot_spec = outer[0])
gs3 = gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec = outer[1])
gs4 = gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec = outer[2])

fig = plt.figure(figsize=(6/inToCm,11/inToCm))
axes = [fig.add_subplot(gs2[i]) for i in range(14)]
axesCB=[fig.add_subplot(gs3[i]) for i in range(1)]

axesSP=fig.add_subplot(outer[2])
axOc=fig.add_subplot(outer[4])

m=np.nanpercentile(avgT,99)
orig_cmap = matplotlib.cm.coolwarm
cmap=shiftedColorMap(orig_cmap,midpoint=1-(m/(m+1)))
trLab=treatName

reorder=[1,0]
for a in treatSimple:
    an=reorder[a]
    for i in range(len(levels)):
        ind=i+(7*a)
        im = axes[ind].imshow(avgT[an,i,:,:],
                              clim=[0,m],
                              extent=[-31,31,-31,31],
                              origin='lower',
                              cmap=cmap,
                             interpolation='none')
        axes[ind].tick_params(axis='y', which='both',length=0)
        axes[ind].tick_params(axis='x', which='both',length=0)
        axes[ind].set_xticks([])
        axes[ind].set_yticks([])
        axes[ind].spines['top'].set_color('white')
        axes[ind].spines['bottom'].set_color('white')
        axes[ind].spines['left'].set_color('white')
        axes[ind].spines['right'].set_color('white')

        if i==0:
            axes[ind].set_ylabel(treatNameSimple[an],color=pal[an],rotation=0, labelpad=20)
            
        if (i==5)&(a==0):
            axes[ind].set_title('Neighbor density')

cbar=plt.colorbar(im,cax=axesCB[0],ticks=np.round([0,1,m-0.2]))
plt.subplots_adjust(wspace=0, hspace=0.1)

social['treatNameSimple']=treatNameSimple[social.group.values.astype('int')]


sns.pointplot(x="xpretty", 
              y="si", 
              hue='treatNameSimple',
              data=social,
              dodge=.3,
              scale=0.5,
              ax=axesSP,
              palette=palSimplePoint,
              linewidth=.5,
              edgecolor='gray',
              linestyles=':',
             ci='sd',
             order=order,
             hue_order=['Control','Ablation'])

sns.stripplot(x="xpretty", 
              y="si", 
              hue='treatNameSimple',
              data=social,
              dodge=.01,
              ax=axesSP,
              palette=palSimplePoint,
             size=1,
             alpha=.1,
             order=order,
             hue_order=['Control','Ablation'])

plt.setp(axesSP.collections, sizes=[20],zorder=-100)

axesSP.spines['bottom'].set_color('white')
axesSP.tick_params(axis='x', which='both',length=0)
axesSP.set_yticks([0,.1,.2,.3,.4])
axesSP.yaxis.tick_left()
axesSP.set_xlabel('')
axesSP.set_ylabel('Attraction')
#handles, labels = axesSP.get_legend_handles_labels()

#axesSP.legend(handles[:2], 
#              labels[:2],
#              loc='upper left',
#              frameon=False,
#             borderaxespad=0.,
#             handletextpad=0.0)

axesSP.legend().remove()

axesSP.axhline(0,ls=':',color='k')

axesSP.text(0-.2,.5,'ns')
axesSP.text(1-.4,.5,'***')
axesSP.text(2-.4,.5,'***')
axesSP.text(3-.4,.5,'***')

axesSP.text(4-.4,.5,'***')
axesSP.text(5-.4,.5,'***')
axesSP.text(6-.4,.5,'***')
axesSP.set_xticks([])
sns.despine()
plt.setp(axesSP.get_xticklabels(), visible=False)

sns.pointplot(data=dfPQ,
             x='xpretty',
             hue='treatNameSimple',
             y='aversion',
             ci='sd',
             dodge=0.3,
              linewidth=.5,
             linestyles=':',
              scale=0.5,
             palette=palSimplePoint,
             order=order,
              estimator=np.median,
              ax=axOc,
             hue_order=['Control','Ablation'])

sns.stripplot(data=dfPQ,
             x='xpretty',
             hue='treatNameSimple',
             y='aversion',
              #ci='sd',
             dodge=0.01,
              size=1,
              palette=palSimplePoint,
             #color='gray',
             alpha=0.1,
             zorder=-1000,
             order=order,
              ax=axOc,
             hue_order=['Control','Ablation'])

plt.setp(axOc.collections, sizes=[20],zorder=-100)

axOc.axhline(0,ls=':',color='gray')
axOc.legend().remove()

axOc.spines['bottom'].set_color('white')
axOc.tick_params(axis='x', which='both',length=0)
axOc.set_xlabel('Dot size (mm)')
axOc.set_ylabel('Short range \nrepulsion (a.u.)')
#axOc.set_yscale('log')
#axOc.set_yticklabels(['','','.1','1','10'])

axOc.text(0-.2,1,'ns')
axOc.text(1-.2,1,'ns')
axOc.text(2-.2,1,'ns')
axOc.text(3-.2,1,'ns')

axOc.text(4-.2,1,'ns')
axOc.text(5-.2,1,'ns')
axOc.text(6-.2,1,'ns')

sns.despine()
plt.rcParams['svg.fonttype'] = 'none'
figPath=FigureDir+'5_MTZ_mapsAndAttractionSimple_AttractAversion.svg'
plt.savefig(figPath,bbox_inches='tight')



In [ ]:
social

In [ ]:
df.head()

In [ ]:
dfPQ

In [ ]:
dfPQm=pd.merge(dfPQ,df[df.epiNr==0][['anIx','animalIndex']],left_on='an',right_on='anIx',copy=False)
dfPQm=pd.merge(social,dfPQm[['animalIndex','xpretty','aversion']],on=['animalIndex','xpretty'])
dfPQm

In [ ]:
dfPQm.to_csv(FigureDir+'SourceDataS7.csv')
FigureDir+'SourceDataS7.csv'

In [ ]:
import matplotlib.gridspec as gridspec
#plt.rcParams['svg.fonttype'] = 'path'

outer = gridspec.GridSpec(3, 2, width_ratios = [5,.1], wspace = 0.05,height_ratios = [1,2,2]) 
#make nested gridspecs
gs2 = gridspec.GridSpecFromSubplotSpec(2, 7, subplot_spec = outer[0])
gs3 = gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec = outer[1])
gs4 = gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec = outer[2])

fig = plt.figure(figsize=(6/inToCm,11/inToCm))
axes = [fig.add_subplot(gs2[i]) for i in range(14)]
axesCB=[fig.add_subplot(gs3[i]) for i in range(1)]

axesSP=fig.add_subplot(outer[2])
axOc=fig.add_subplot(outer[4])

m=np.nanpercentile(avgT,99)
orig_cmap = matplotlib.cm.coolwarm
cmap=shiftedColorMap(orig_cmap,midpoint=1-(m/(m+1)))
trLab=treatName

reorder=[1,0]
for a in treatSimple:
    an=reorder[a]
    for i in range(len(levels)):
        ind=i+(7*a)
        im = axes[ind].imshow(avgT[an,i,:,:],
                              clim=[0,m],
                              extent=[-31,31,-31,31],
                              origin='lower',
                              cmap=cmap,
                             interpolation='none')
        axes[ind].tick_params(axis='y', which='both',length=0)
        axes[ind].tick_params(axis='x', which='both',length=0)
        axes[ind].set_xticks([])
        axes[ind].set_yticks([])
        axes[ind].spines['top'].set_color('white')
        axes[ind].spines['bottom'].set_color('white')
        axes[ind].spines['left'].set_color('white')
        axes[ind].spines['right'].set_color('white')

        if i==0:
            axes[ind].set_ylabel(treatNameSimple[an],color=pal[an],rotation=0, labelpad=20)
            
        if (i==5)&(a==0):
            axes[ind].set_title('Neighbor density')

cbar=plt.colorbar(im,cax=axesCB[0],ticks=np.round([0,1,m-0.2]))
plt.subplots_adjust(wspace=0, hspace=0.1)

social['treatNameSimple']=treatNameSimple[social.group.values.astype('int')]


sns.pointplot(x="xpretty", 
              y="si", 
              hue='treatNameSimple',
              data=dfPQm,
              dodge=.3,
              scale=0.5,
              ax=axesSP,
              palette=palSimplePoint,
              linewidth=.5,
              edgecolor='gray',
              linestyles=':',
             ci='sd',
             order=order,
             hue_order=['Control','Ablation'])

sns.stripplot(x="xpretty", 
              y="si", 
              hue='treatNameSimple',
              data=dfPQm,
              dodge=.01,
              ax=axesSP,
              palette=palSimplePoint,
             size=1,
             alpha=.1,
             order=order,
             hue_order=['Control','Ablation'])

plt.setp(axesSP.collections, sizes=[20],zorder=-100)

axesSP.spines['bottom'].set_color('white')
axesSP.tick_params(axis='x', which='both',length=0)
axesSP.set_yticks([0,.1,.2,.3,.4])
axesSP.yaxis.tick_left()
axesSP.set_xlabel('')
axesSP.set_ylabel('Attraction')
#handles, labels = axesSP.get_legend_handles_labels()

#axesSP.legend(handles[:2], 
#              labels[:2],
#              loc='upper left',
#              frameon=False,
#             borderaxespad=0.,
#             handletextpad=0.0)

axesSP.legend().remove()

axesSP.axhline(0,ls=':',color='k')

axesSP.text(0-.2,.5,'ns')
axesSP.text(1-.4,.5,'**')
axesSP.text(2-.4,.5,'***')
axesSP.text(3-.4,.5,'***')

axesSP.text(4-.4,.5,'***')
axesSP.text(5-.4,.5,'***')
axesSP.text(6-.4,.5,'***')
axesSP.set_xticks([])
sns.despine()
plt.setp(axesSP.get_xticklabels(), visible=False)

sns.pointplot(data=dfPQm,
             x='xpretty',
             hue='treatNameSimple',
             y='aversion',
             ci='sd',
             dodge=0.3,
              linewidth=.5,
             linestyles=':',
              scale=0.5,
             palette=palSimplePoint,
             order=order,
              estimator=np.median,
              ax=axOc,
             hue_order=['Control','Ablation'])

sns.stripplot(data=dfPQm,
             x='xpretty',
             hue='treatNameSimple',
             y='aversion',
              #ci='sd',
             dodge=0.01,
              size=1,
              palette=palSimplePoint,
             #color='gray',
             alpha=0.1,
             zorder=-1000,
             order=order,
              ax=axOc,
             hue_order=['Control','Ablation'])

plt.setp(axOc.collections, sizes=[20],zorder=-100)

axOc.axhline(0,ls=':',color='gray')
axOc.legend().remove()

axOc.spines['bottom'].set_color('white')
axOc.tick_params(axis='x', which='both',length=0)
axOc.set_xlabel('Dot size (mm)')
axOc.set_ylabel('Short range \nrepulsion (a.u.)')
#axOc.set_yscale('log')
#axOc.set_yticklabels(['','','.1','1','10'])

axOc.text(0-.2,1,'ns')
axOc.text(1-.2,1,'ns')
axOc.text(2-.2,1,'ns')
axOc.text(3-.2,1,'ns')

axOc.text(4-.2,1,'ns')
axOc.text(5-.2,1,'ns')
axOc.text(6-.2,1,'ns')

sns.despine()
plt.rcParams['svg.fonttype'] = 'none'
figPath=FigureDir+'5_MTZ_mapsAndAttractionSimple_AttractAversion.svg'
plt.savefig(figPath,bbox_inches='tight')



In [ ]:
import matplotlib.gridspec as gridspec
#plt.rcParams['svg.fonttype'] = 'path'

outer = gridspec.GridSpec(3, 2, width_ratios = [5,.1], wspace = 0.05,height_ratios = [1,2,2]) 
#make nested gridspecs
gs2 = gridspec.GridSpecFromSubplotSpec(2, 7, subplot_spec = outer[0])
gs3 = gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec = outer[1])
gs4 = gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec = outer[2])

fig = plt.figure(figsize=(6/inToCm,11/inToCm))
axes = [fig.add_subplot(gs2[i]) for i in range(14)]
axesCB=[fig.add_subplot(gs3[i]) for i in range(1)]

axesSP=fig.add_subplot(outer[2])
axOc=fig.add_subplot(outer[4])

m=np.nanpercentile(avgT,99)
orig_cmap = matplotlib.cm.coolwarm
cmap=shiftedColorMap(orig_cmap,midpoint=1-(m/(m+1)))
trLab=treatName

reorder=[1,0]
for a in treatSimple:
    an=reorder[a]
    for i in range(len(levels)):
        ind=i+(7*a)
        im = axes[ind].imshow(avgT[an,i,:,:],
                              clim=[0,m],
                              extent=[-31,31,-31,31],
                              origin='lower',
                              cmap=cmap,
                             interpolation='none')
        axes[ind].tick_params(axis='y', which='both',length=0)
        axes[ind].tick_params(axis='x', which='both',length=0)
        axes[ind].set_xticks([])
        axes[ind].set_yticks([])
        axes[ind].spines['top'].set_color('white')
        axes[ind].spines['bottom'].set_color('white')
        axes[ind].spines['left'].set_color('white')
        axes[ind].spines['right'].set_color('white')

        if i==0:
            axes[ind].set_ylabel(treatNameSimple[an],color=pal[an],rotation=0, labelpad=20)
            
        if (i==5)&(a==0):
            axes[ind].set_title('Neighbor density')

cbar=plt.colorbar(im,cax=axesCB[0],ticks=np.round([0,1,m-0.2]))
plt.subplots_adjust(wspace=0, hspace=0.1)

social['treatNameSimple']=treatNameSimple[social.group.values.astype('int')]


sns.pointplot(x="xpretty", 
              y="si", 
              hue='treatNameSimple',
              data=dfPQm,
              dodge=.3,
              scale=0.5,
              ax=axesSP,
              palette=palSimplePoint,
              linewidth=.5,
              edgecolor='gray',
              linestyles=':',
             ci='sd',
             order=order,
             hue_order=['Control','Ablation'])

sns.stripplot(x="xpretty", 
              y="si", 
              hue='treatNameSimple',
              data=dfPQm,
              dodge=.01,
              ax=axesSP,
              palette=palSimplePoint,
             size=1,
             alpha=.1,
             order=order,
             hue_order=['Control','Ablation'])

plt.setp(axesSP.collections, sizes=[20],zorder=-100)

axesSP.spines['bottom'].set_color('white')
axesSP.tick_params(axis='x', which='both',length=0)
axesSP.set_yticks([0,.1,.2,.3,.4])
axesSP.yaxis.tick_left()
axesSP.set_xlabel('')
axesSP.set_ylabel('Attraction')
#handles, labels = axesSP.get_legend_handles_labels()

#axesSP.legend(handles[:2], 
#              labels[:2],
#              loc='upper left',
#              frameon=False,
#             borderaxespad=0.,
#             handletextpad=0.0)

axesSP.legend().remove()

axesSP.axhline(0,ls=':',color='k')

axesSP.text(0-.2,.5,'ns')
axesSP.text(1-.4,.5,'**')
axesSP.text(2-.4,.5,'***')
axesSP.text(3-.4,.5,'***')

axesSP.text(4-.4,.5,'***')
axesSP.text(5-.4,.5,'***')
axesSP.text(6-.4,.5,'***')
axesSP.set_xticks([])
sns.despine()
plt.setp(axesSP.get_xticklabels(), visible=False)

sns.pointplot(data=dfPQm,
             x='xpretty',
             hue='treatNameSimple',
             y='aversion',
             ci='sd',
             dodge=0.3,
              linewidth=.5,
             linestyles=':',
              scale=0.5,
             palette=palSimplePoint,
             order=order,
              estimator=np.median,
              ax=axOc,
             hue_order=['Control','Ablation'])

sns.stripplot(data=dfPQm,
             x='xpretty',
             hue='treatNameSimple',
             y='aversion',
              #ci='sd',
             dodge=0.01,
              size=1,
              palette=palSimplePoint,
             #color='gray',
             alpha=0.1,
             zorder=-1000,
             order=order,
              ax=axOc,
             hue_order=['Control','Ablation'])

plt.setp(axOc.collections, sizes=[20],zorder=-100)

axOc.axhline(0,ls=':',color='gray')
axOc.legend().remove()

axOc.spines['bottom'].set_color('white')
axOc.tick_params(axis='x', which='both',length=0)
axOc.set_xlabel('Dot size (mm)')
axOc.set_ylabel('Short range \nrepulsion (a.u.)')
#axOc.set_yscale('log')
#axOc.set_yticklabels(['','','.1','1','10'])

axOc.text(0-.2,1,'ns')
axOc.text(1-.2,1,'ns')
axOc.text(2-.2,1,'ns')
axOc.text(3-.2,1,'ns')

axOc.text(4-.2,1,'ns')
axOc.text(5-.2,1,'ns')
axOc.text(6-.2,1,'ns')

sns.despine()
plt.rcParams['svg.fonttype'] = 'none'
figPath=FigureDir+'5_MTZ_mapsAndAttractionSimple_AttractAversion.svg'
plt.savefig(figPath,bbox_inches='tight')



In [ ]:
for i,x in enumerate(dfPQm.xpretty.unique()):
    ix=(dfPQm.xpretty==x)&(dfPQm['treatNameSimple']=='Ablation')
    a=dfPQm[ix]['si']
    ix=(dfPQm.xpretty==x)&(dfPQm['treatNameSimple']=='Control')
    b=dfPQm[ix]['si']
    s,p=stats.ttest_ind(a,b)
    print(s,',',p)

In [ ]:
for i,x in enumerate(dfPQm.xpretty.unique()):
    ix=(dfPQm.xpretty==x)&(dfPQm['treatNameSimple']=='Ablation')
    a=dfPQm[ix]['aversion']
    ix=(dfPQm.xpretty==x)&(dfPQm['treatNameSimple']=='Control')
    b=dfPQm[ix]['aversion']
    s,p=stats.ttest_ind(a,b)
    print(s,',',p)

In [ ]:


fig,axOc=plt.subplots(figsize=(6/inToCm,4.5/inToCm))
sns.pointplot(data=dfPQ,
             x='xpretty',
             hue='treatNameSimple',
             y='aversion',
             ci='sd',
             dodge=0.3,
              linewidth=.5,
             linestyles=':',
              scale=0.5,
             palette=palSimplePoint,
             order=order,
              estimator=np.median,
              ax=axOc,
             hue_order=['Control','Ablation'])

sns.stripplot(data=dfPQ,
             x='xpretty',
             hue='treatNameSimple',
             y='aversion',
              #ci='sd',
             dodge=0.01,
              size=1,
              palette=palSimplePoint,
             #color='gray',
             alpha=0.1,
             zorder=-1000,
             order=order,
              ax=axOc,
              clip_on = False,
             hue_order=['Control','Ablation'])

plt.setp(axOc.collections, sizes=[20],zorder=-100)

axOc.axhline(0,ls=':',color='gray')
axOc.legend().remove()

#axOc.spines['bottom'].set_color('white')
axOc.tick_params(axis='x', which='both',length=0)
axOc.set_xlabel('Dot size (mm)')
axOc.set_ylabel('Short range \nrepulsion (a.u.)')
#axOc.set_yscale('log')
#axOc.set_yticklabels(['','','.1','1','10'])

axOc.text(0-.2,1.3,'ns')
axOc.text(1-.2,1.3,'ns')
axOc.text(2-.2,1.3,'ns')
axOc.text(3-.2,1.3,'ns')

axOc.text(4-.2,1.3,'ns')
axOc.text(5-.2,1.3,'ns')
axOc.text(6-.2,1.3,'ns')
axOc.set_ylim(0,1.5)
sns.despine()
plt.rcParams['svg.fonttype'] = 'none'
figPath=FigureDir+'S7g.svg'
plt.savefig(figPath,bbox_inches='tight')